In [1]:
from bs4 import BeautifulSoup as BS
import pandas as pd
import requests
import webbrowser
import urllib.request
def getReviews(url):
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0')
    req.add_header('Accept', 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,/;q=0.8')
    req.add_header('Accept-Language', 'en-US,en;q=0.5')
    
    response = urllib.request.urlopen(req)
    html_code = response.read().decode('utf-8')
    soup = BS(html_code, 'html.parser')

    # Check for total reviews
    totalReviews_element = soup.find('span', class_='biGQs _P pZUbB KxBGd')
    totalReviews = totalReviews_element.text[:-8].replace(',', '') if totalReviews_element else "0"

    # Hotel Name
    hotelName_element = soup.find('h1', id='HEADING')
    hotelName = hotelName_element.text if hotelName_element else "Unknown Hotel"

    # Review Heads
    reviewHeads = [i.text for i in soup.find_all('span', class_='JbGkU Cj')]

    # Review Content
    reviewContent = [i.text for i in soup.find_all('span', class_='orRIx Ci _a C')]

    # Reviewer Name and Date
    reviewerNdate = [i.text for i in soup.find_all('div', class_='tVWyV _Z o S4 H3 Ci') if 'wrote a review' in i.text]
    reviewerandDate = []
    for i in reviewerNdate:
        reviewerandDate.append(i.split('wrote a review '))

    finalDate = []
    finalreviewer = []
    for i in reviewerandDate:
        finalDate.append(i[1] if len(i) > 1 else "Unknown Date")
        finalreviewer.append(i[0] if len(i) > 0 else "Unknown Reviewer")

    # Ratings
    ratings = [i.find('title').text[0:3] for i in soup.find_all('div', class_='kmMXA _T Gi')]

    # Hotel Location
    tempHotelLocation = [i.text for i in soup.find_all('span', class_='biGQs _P pZUbB KxBGd')]
    hotelLocation = tempHotelLocation[1] if len(tempHotelLocation) > 1 else "Unknown Location"

    # Create DataFrame
    df = pd.DataFrame({
        'Hotel Name': hotelName,
        'Hotel Location': hotelLocation,
        'Reviewer': finalreviewer,
        'Rating': ratings,
        'Date of Review': finalDate,
        'Review': reviewHeads,
        'Complete Review': reviewContent
    })

    return [df, totalReviews, hotelName, hotelLocation]


def getPureReviews(url,hotelName,hotelLocation):
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0')
    req.add_header('Accept', 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,/;q=0.8')
    req.add_header('Accept-Language', 'en-US,en;q=0.5')
    response = urllib.request.urlopen(req)
    html_code = response.read().decode('utf-8')
    soup=BS(html_code,'html.parser')
    reviewHeads=[i.text for i in soup.find_all('span',class_='JbGkU Cj')]
    reviewContent=[i.text for i in soup.find_all('span',class_='orRIx Ci _a C')]
    reviewerNdate = [i.text for i in soup.find_all('div', class_='tVWyV _Z o S4 H3 Ci') if 'wrote a review' in i.text]
    reviewerandDate=[]
    for i in reviewerNdate:
        reviewerandDate.append(i.split('wrote a review '))
    finalDate=[]
    finalreviewer=[]
    for i in reviewerandDate:
        finalDate.append(i[1])
        finalreviewer.append(i[0])
    ratings=[i.find('title').text[0:3] for i in soup.find_all('div',class_='kmMXA _T Gi')]
    df = pd.DataFrame({
        'Hotel Name': hotelName,
        'Hotel Location': hotelLocation,
        'Reviewer': finalreviewer,
        'Rating':ratings,
        'Date of Review': finalDate,
        'Review':reviewHeads,
        'Complete Review':reviewContent
    })
    return df

In [3]:
url='https://www.tripadvisor.in/Restaurant_Review-g297586-d19565060-Reviews-Turquoise-Hyderabad_Hyderabad_District_Telangana.html'
split_url = url.split("Reviews-")
first_part = split_url[0] + "Reviews-or"
second_part = "-" + split_url[1]
result = getReviews(url)
df=result[0]
totalReviews=result[1]
hotelName=result[2]
hotelLocation=result[3]
urls = [f"{first_part}{i}{second_part}" for i in range(10, int(totalReviews)-10, 10)]
for i in urls:
    newdf=getPureReviews(i,hotelName,hotelLocation)
    df = pd.concat([df, newdf], ignore_index=True)
df

,Hotel Name,Hotel Location,Reviewer,Rating,Date of Review,Review,Complete Review


In [4]:
df.to_excel('trial1.xlsx', index=False, engine='openpyxl')

In [5]:
url='https://www.tripadvisor.in/Hotel_Review-g297698-d304528-Reviews-The_Westin_Resort_Nusa_Dua_Bali-Nusa_Dua_Benoa_South_Kuta_Bali.html'
df=getReviews(url)
df[0]

,Hotel Name,Hotel Location,Reviewer,Rating,Date of Review,Review,Complete Review
0,The Westin Resort Nusa Dua Bali,"Kawasan Pariwisata Nusa Dua, Itdc Heavenly Spa...",Valdis,5.0,Jul 2020,Long term experience!,I stayed in Westin Nusa Dua for almost 2 month...
1,The Westin Resort Nusa Dua Bali,"Kawasan Pariwisata Nusa Dua, Itdc Heavenly Spa...",Benny L,5.0,Sept 2020,Amazing staycation,We had a chance to take a short break for 2 ni...
2,The Westin Resort Nusa Dua Bali,"Kawasan Pariwisata Nusa Dua, Itdc Heavenly Spa...",tommydju,5.0,Aug 2020,Excellent Service and Asistance by Westin Hote...,This is my first vacation after pandemic ..I c...
3,The Westin Resort Nusa Dua Bali,"Kawasan Pariwisata Nusa Dua, Itdc Heavenly Spa...",jason245,3.0,Jan 2020,"Very Quiet, Lacks atmosphere",Checked in for 7 nights with 2 adjoining rooms...
4,The Westin Resort Nusa Dua Bali,"Kawasan Pariwisata Nusa Dua, Itdc Heavenly Spa...",Fatima Essop,5.0,Jan 2020,Beautiful spacious suite & fantastic service,We are returning guests to the Westin Nusa Dua...
5,The Westin Resort Nusa Dua Bali,"Kawasan Pariwisata Nusa Dua, Itdc Heavenly Spa...",Utami,5.0,Aug 2020,Amazing!,Not sure where to start as everything was awes...
6,The Westin Resort Nusa Dua Bali,"Kawasan Pariwisata Nusa Dua, Itdc Heavenly Spa...",E Rizaldy,5.0,Mar 2021,Excellent Weekend Escape,We stayed in Westin for 4 nights in last week ...
7,The Westin Resort Nusa Dua Bali,"Kawasan Pariwisata Nusa Dua, Itdc Heavenly Spa...",Natalia K,5.0,Jan 2020,Very pleasant and relaxing,I would recommend this hotel to families with ...
8,The Westin Resort Nusa Dua Bali,"Kawasan Pariwisata Nusa Dua, Itdc Heavenly Spa...",1849jack,5.0,Jan 2020,Family Holiday,We stayed in the Family Suite and we are famil...
9,The Westin Resort Nusa Dua Bali,"Kawasan Pariwisata Nusa Dua, Itdc Heavenly Spa...",Brett from Melbourne,5.0,Mar 2020,Nature Features Heavily,What a delightful resort! After navigating the...


In [8]:
df

[                        Hotel Name  \
 0  The Westin Resort Nusa Dua Bali   
 1  The Westin Resort Nusa Dua Bali   
 2  The Westin Resort Nusa Dua Bali   
 3  The Westin Resort Nusa Dua Bali   
 4  The Westin Resort Nusa Dua Bali   
 5  The Westin Resort Nusa Dua Bali   
 6  The Westin Resort Nusa Dua Bali   
 7  The Westin Resort Nusa Dua Bali   
 8  The Westin Resort Nusa Dua Bali   
 9  The Westin Resort Nusa Dua Bali   
 
                                       Hotel Location               Reviewer  \
 0  Kawasan Pariwisata Nusa Dua, Itdc Heavenly Spa...                Valdis    
 1  Kawasan Pariwisata Nusa Dua, Itdc Heavenly Spa...               Benny L    
 2  Kawasan Pariwisata Nusa Dua, Itdc Heavenly Spa...              tommydju    
 3  Kawasan Pariwisata Nusa Dua, Itdc Heavenly Spa...              jason245    
 4  Kawasan Pariwisata Nusa Dua, Itdc Heavenly Spa...          Fatima Essop    
 5  Kawasan Pariwisata Nusa Dua, Itdc Heavenly Spa...                 Utami    
 6  Kawas

In [13]:
import csv

In [ ]:
url='https://www.tripadvisor.in/Hotel_Review-g297698-d304528-Reviews-The_Westin_Resort_Nusa_Dua_Bali-Nusa_Dua_Benoa_South_Kuta_Bali.html'
df=getReviews(url)
df[0